In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn 
from torch.utils.data import DataLoader     # 데이터로더는 데이터셋을 iterable하게 감싸는 역할
from torchvision import datasets            # 데이터셋은 샘플과 정답을 저장함
from torchvision.transforms import ToTensor
import clip
from PIL import Image

## CLIP model

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [3]:
image_title = 'A horse in the space.png'

prompts = [
    "A horse in the space",
    "A dog in the space",
    "A bear in the space",
    "A person in the space",
    "A horse in the park",
    "A dog in the park",
    "A bear in the park",
    "A person in the park",
]

image = preprocess(Image.open(image_title)).unsqueeze(0).to(device)
text = clip.tokenize(prompts).to(device)

In [30]:
model(image, text)

(tensor([[30.6719, 24.0625, 22.5156, 23.5312, 23.3750, 16.5312, 14.7188, 17.2344]],
        device='cuda:0', dtype=torch.float16, grad_fn=<MmBackward0>),
 tensor([[30.6719],
         [24.0625],
         [22.5156],
         [23.5312],
         [23.3750],
         [16.5312],
         [14.7188],
         [17.2344]], device='cuda:0', dtype=torch.float16, grad_fn=<TBackward0>))

## Our Model

In [60]:
def get_image_features_per_layer(clip_model, image):
    features = [torch.zeros([image.shape[0], 768]) for i in range(clip_model.visual.transformer.layers)]

    # image -> tokens
    x = clip_model.visual.conv1(image.type(clip_model.visual.conv1.weight.dtype))
    x = x.reshape(x.shape[0], x.shape[1], -1)   # shape = [*, width, grid ** 2]
    x = x.permute(0, 2, 1)                      # shape = [*, grid ** 2, width]
    x = torch.cat([clip_model.visual.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x], dim=1)  # shape = [*, grid ** 2 + 1, width]
    x = x + clip_model.visual.positional_embedding.to(x.dtype)
    x = clip_model.visual.ln_pre(x)

    # tokens -> transformer -> feature_embeddings
    x = x.permute(1, 0, 2)          # NLD -> LND

    for i in range(clip_model.visual.transformer.layers):
        x = clip_model.visual.transformer.resblocks[i](x)
        tmp = x.permute(1, 0, 2)    # LND -> NLD
        tmp = tmp[:, 0, :]
        features[i].copy_(tmp)

    tensor = torch.stack(features, dim=0).detach()
    # tensor.requires_grad = False

    return tensor.to(device)

In [42]:
def get_text_features_per_layer(clip_model, text):
    features = [torch.zeros([*text.shape, 512]) for i in range(clip_model.transformer.layers)]

    #%% clip_model.encode_image

    x = clip_model.token_embedding(text).type(clip_model.dtype)  # [batch_size, n_ctx, d_clip_model]

    x = x + clip_model.positional_embedding.type(clip_model.dtype)
    x = x.permute(1, 0, 2)          # NLD -> LND

    for i in range(clip_model.transformer.layers):
        x = clip_model.transformer.resblocks[i](x)
        tmp = x.permute(1, 0, 2)    # LND -> NLD
        features[i].copy_(tmp)

    tensor = torch.stack(features, dim=0).detach()
    # tensor.requires_grad = False

    return tensor.to(device)

#### lab

In [43]:
model.visual.ln_post(get_image_features_per_layer(model, image).to(device)).shape

torch.Size([12, 1, 768])

In [44]:
model.ln_final

LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [34]:
model.ln_final(get_text_features_per_layer(model, text).to(device)).shape

torch.Size([12, 8, 77, 512])

In [35]:
model.visual.proj.shape

torch.Size([768, 512])

In [36]:
torch.rand(model.visual.proj.shape)

tensor([[0.7060, 0.3264, 0.8114,  ..., 0.3201, 0.6791, 0.0149],
        [0.8198, 0.7222, 0.8461,  ..., 0.4048, 0.2797, 0.4489],
        [0.4857, 0.4051, 0.6350,  ..., 0.2085, 0.0135, 0.5073],
        ...,
        [0.2146, 0.3847, 0.2523,  ..., 0.9563, 0.4808, 0.8936],
        [0.9526, 0.3367, 0.8129,  ..., 0.4689, 0.7941, 0.3421],
        [0.1183, 0.8862, 0.1054,  ..., 0.7301, 0.8447, 0.9090]])

In [37]:
proj = torch.stack([model.visual.proj for _ in range(12)])
proj.shape

torch.Size([12, 768, 512])

In [38]:
tmp = [model.visual.proj for _ in range(12)]
tmp[0] is tmp[1], proj[0] is proj[1]

(True, False)

In [39]:
print(proj[0][0][0].item(), proj[1][0][0].item())
proj[0][0][0] = -1
print(proj[0][0][0].item(), proj[1][0][0].item())
# light copy

-0.0026264190673828125 -0.0026264190673828125
-1.0 -0.0026264190673828125


In [42]:
tmp = get_image_features_per_layer(model, image).to(device)
tmp1 = model.visual.ln_post(tmp)[0]
tmp2 = model.visual.ln_post(tmp[0])
(tmp1 == tmp2).sum()

tensor(768, device='cuda:0')

#### inference

In [357]:
# image_features.shape    # (12, 1, 768)
# model.visual_projection[0].shape    # (768, 512)
# (image_features @ model.visual_projection[0]).shape # (12, 1, 512)


torch.Size([12, 1, 512])

In [426]:
model.parameters
# optimizer를 정의하고 파라미터를 

<bound method Module.parameters of OurCLIP(
  (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (ln_final): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)>

In [418]:
# Define model
class OurCLIP(nn.Module):        
    def __init__(self, clip_model, # use pre-trained clip model
                 use_one_ln1=True, use_one_ln2=True, 
                 use_one_projection1=True, use_one_projection2=True,
                 trainable_ln1=False, trainable_ln2=False, 
                 trainable_projection1=False, trainable_projection2=False,
                 threshold = 20, score_type=0): 
        super().__init__()

        self.dtype = clip_model.dtype
        self.logit_scale = clip_model.logit_scale
        self.threshold = threshold
        self.score_type = score_type

        ####################### 미구현
        if use_one_ln1:
            self.ln_post = clip_model.visual.ln_post.requires_grad_(False)
        else: pass
        if use_one_ln2:
            self.ln_final = clip_model.ln_final.requires_grad_(False)
        else: pass
        ########################

        if use_one_projection1:
            # nn.Parameters() 사용해보기
            self.visual_projection = clip_model.visual.proj.detach().requires_grad_(True).type(torch.float)
            if not trainable_projection1:
                self.visual_projection = clip_model.visual.proj.detach().requires_grad_(False).type(torch.float)
        else:
            self.visual_projection = torch.stack([*[torch.rand(clip_model.visual.proj.shape).to(device) for _ in range(11)],
                                                   clip_model.visual.proj]).detach().requires_grad_(True).type(torch.float)
        
        if use_one_projection2:
            self.textual_projection = clip_model.text_projection.detach().requires_grad_(True)
            if not trainable_projection2:
                self.textual_projection = clip_model.text_projection.detach().requires_grad_(False)
        else:
            self.textual_projection = torch.stack([*[torch.rand(clip_model.text_projection.shape).to(device) for _ in range(11)], 
                                                   clip_model.text_projection]).detach().requires_grad_(True)


    def forward(self, image_features, text, text_features):    
        # image_features = image_features.to(device)
        # text_features = text_features.to(device)
    
        image_features = self.ln_post(image_features)                   # (12, batch_size, 768)
        image_embeddings = image_features @ self.visual_projection      # (12, batch_size, 512)  <- we'll use it

        text_features = self.ln_final(text_features).type(self.dtype)   # (12, seq_len, 77, 512)
        text_embeddings = text_features[:, torch.arange(text_features.shape[1]), text.argmax(dim=-1)].type(torch.float) @ self.textual_projection .type(torch.float) 
                                                                        # (12, seq_len, 512)  <- we'll use it


        # print(text_embeddings.shape, image_embeddings.shape)
        t1 = image_embeddings.permute(1, 0, 2)[0].type(torch.float)     # 일단 1 크기의 배치만을 가정한다.
        t2 = text_embeddings.permute(1, 0, 2).type(torch.float)
        
        t1 = t1 / t1.norm(dim=1, keepdim=True)
        t2 = t2 / t2.norm(dim=1, keepdim=True)
        t3 = (t2 @ t1.T).view([t2.shape[0], -1])

        if self.score_type == 1:
            return torch.mean(t3, 1)    # score2
        if self.score_type == 2:
            return torch.mean(t3, 1)    # score3, count는 일단 미구현
        return torch.max(t3, 1)         # score1     

        ##########################################################################
        #   여러 개의 배치를 사용하는 연산은 생각해내기가 너무 어려움
        #   각 계산한 결과 텐서를 리스트에 따로 저장하고 가져와서 쓰는 것이 grad 계산이 가능할까?
        #   일단 하나의 배치와 하나의 score를 사용하는 것으로 진행하였다.
        ##########################################################################

        scores = [[[], [], []] for _ in range(image_features.shape[1])]

        for k in range(image_features.shape[1]):
            for _class in range(text_features.shape[1]):
                # print(image_embeddings[0].shape)
                # print(text_embeddings[0][[_class]].T.shape)
                # print((image_embeddings[0] @ text_embeddings[0][[_class]].T.type(torch.float)).item())
                img_similarity_by_layer = [[(image_embeddings[i][[k]] @ text_embeddings[j][[_class]].T.type(torch.float))
                                            for j in range(12)] for i in range(12)]
                scores[k][0].append(np.max(img_similarity_by_layer))
                scores[k][1].append(np.mean(img_similarity_by_layer))
                scores[k][2].append(sum(list(map(lambda x: sum([y > self.threshold for y in x]), img_similarity_by_layer))))

        return scores

In [364]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

model = OurCLIP(clip_model, trainable_projection1=True, trainable_projection2=True).to(device)  # model을 initialize하는 부분
print(model)

OurCLIP(
  (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (ln_final): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)


### lab

In [74]:
image_title = 'A horse in the space.png'

prompts = [
    "A horse in the space",
    "A dog in the space",
    "A bear in the space",
    "A person in the space",
    "A horse in the park",
    "A dog in the park",
    "A bear in the park",
    "A person in the park",
]

image = preprocess(Image.open(image_title)).unsqueeze(0).to(device)
text = clip.tokenize(prompts).to(device)

In [75]:
image_features = get_image_features_per_layer(clip_model, image)
text_features = get_text_features_per_layer(clip_model, text)

In [49]:
model(image_features, text_features)

[[33.82638168334961,
  26.978994369506836,
  26.948450088500977,
  30.468639373779297,
  25.630268096923828,
  20.795486450195312,
  22.559879302978516,
  23.461559295654297],
 [12.221618531478775,
  12.280218217107985,
  12.698671211798986,
  14.021323535177443,
  11.308028957910008,
  10.396428364846441,
  11.075213006801075,
  10.72875845928987],
 [14, 13, 13, 13, 6, 1, 6, 11]]

In [50]:
clip_model.text_projection.shape

torch.Size([512, 512])

## Train

### Load-Dataset

In [51]:
import deeplake
from PIL import Image
import numpy as np
import os, time
import torch
from torchvision import transforms, models

In [362]:
# Connect to the training and testing datasets
ds_train = deeplake.load('hub://activeloop/pacs-train')[:5000]  ########## 크기를 줄여 학습을 빠르게 진행하였음
ds_test = deeplake.load('hub://activeloop/pacs-test')[:1000]

tform = transforms.Compose([
    transforms.RandomRotation(20), # Image augmentation
    transforms.ToTensor(), # Must convert to pytorch tensor for subsequent operations to run
    transforms.Normalize([0.5], [0.5]),
])
batch_size = 1

hub://activeloop/pacs-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/pacs-train
hub://activeloop/pacs-test loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/pacs-test


In [331]:
# Specity the loss function and optimizer
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.1)

In [424]:
model.parameters

<bound method Module.parameters of OurCLIP(
  (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (ln_final): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)>

In [282]:
pacs_class = [
    'a dog', 'an elephant', 'a giraffe', 'a guitar', 'a horse', 'a house', 'a person'
]

def prompt(idx):
    return f"An image of {pacs_class[idx]}"

prompts = clip.tokenize([prompt(x) for x in range(7)]).to(device)
print(prompts.shape)

torch.Size([7, 77])


In [283]:
# Since torchvision transforms expect PIL images, we use the 'pil' decode_method for the 'images' tensor. This is much faster than running ToPILImage inside the transform
train_loader = ds_train.pytorch(num_workers = 0, shuffle = True, 
                                transform = {'images': tform, 'labels': None}, 
                                batch_size = batch_size, decode_method = {'images': 'pil'})
test_loader = ds_test.pytorch(num_workers = 0, transform = {'images': tform, 'labels': None}, 
                                batch_size = batch_size, decode_method = {'images': 'pil'})

In [401]:
def train_one_epoch(model, optimizer, data_loader, device, prompts):

    model.train()

    # Zero the performance stats for each epoch
    running_loss = 0.0
    start_time = time.time()
    total = 0
    correct = 0
    
    for i, data in enumerate(data_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['images']
        labels = torch.squeeze(data['labels'])

        inputs = inputs.to(device)
        labels = labels.to(device)

        image_features = get_image_features_per_layer(clip_model, inputs)
        text_features = get_text_features_per_layer(clip_model, prompts)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        if model.score_type == 0:
            output = model(image_features, prompts, text_features)[0]
        elif model.score_type == 1:
            output = model(image_features, prompts, text_features)
        else: assert("미구현")

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        predicted = torch.argmax(output)
        # print(i, '='*5, predicted, labels)
        total += 1  ########################## labels.size(0)
        correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
    
        # Print performance statistics
        running_loss += loss.item()
        if i % 10 == 0:    # print every 10 batches
            batch_time = time.time()
            speed = (i+1)/(batch_time-start_time)
            print('[%5d] loss: %.3f, speed: %.2f, accuracy: %.2f %%' %
                  (i, running_loss, speed, accuracy))

            running_loss = 0.0
            total = 0
            correct = 0
            # break
        if i != 0 and i % 3000 == 0:
            break;

    
def test_model(model, data_loader):

    model.eval()

    start_time = time.time()
    total = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['images']
            labels = torch.squeeze(data['labels'])

            inputs = inputs.to(device)
            labels = labels.to(device)

            image_features = get_image_features_per_layer(clip_model, inputs)
            text_features = get_text_features_per_layer(clip_model, prompts)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            if model.score_type == 0:
                output = model(image_features, prompts, text_features)[0]
            elif model.score_type == 1:
                output = model(image_features, prompts, text_features)
            else: assert("미구현")
            # outputs = torch.Tensor(list(map(lambda x: x[0], output))).to(device)

            predicted = torch.argmax(output)
            total += 1 ########################## labels.size(0)
            correct += (predicted == labels).sum().item()
        
            if i != 0 and i % 100 == 0:
                break
        accuracy = 100 * correct / total
            
        print('Finished Testing')
        print('Testing accuracy: %.1f %%' %(accuracy))

### lab

In [289]:
data = next(iter(train_loader))
inputs = data['images']
labels = torch.squeeze(data['labels'])

inputs = inputs.to(device)
labels = labels.to(device)
labels

tensor(2, device='cuda:0')

In [287]:
inputs.shape, prompts.shape
get_image_features_per_layer(clip_model, inputs).shape, get_text_features_per_layer(clip_model, prompts).shape

(torch.Size([12, 1, 768]), torch.Size([12, 7, 77, 512]))

In [107]:
image_features = get_image_features_per_layer(clip_model, inputs)
text_features = get_text_features_per_layer(clip_model, prompts)

In [290]:
output = model(image_features, prompts, text_features)
output

torch.return_types.max(
values=tensor([1.3213, 1.2870, 0.9971, 0.9502, 1.3020, 1.6231, 1.1956],
       device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([141, 139, 139, 138, 139, 143, 142], device='cuda:0'))

In [126]:
# outputs = torch.Tensor(list(map(lambda x: x[0], scores))).to(device)
# outputs, labels

(tensor([[33.2395, 28.6383, 27.9835, 37.3564, 32.7737, 30.8021, 35.0590],
         [33.6642, 28.3727, 27.1275, 25.2947, 40.4309, 29.3082, 34.7931],
         [33.8334, 29.4328, 28.4747, 37.0757, 34.1582, 29.9939, 36.0595],
         [33.4167, 37.2045, 26.9609, 24.3298, 32.5465, 29.5641, 35.3329]],
        device='cuda:0'),
 tensor([3, 4, 3, 1], device='cuda:0'))

In [292]:
loss = criterion(output[0], labels)
loss

tensor(2.2104, device='cuda:0', grad_fn=<NllLossBackward0>)

In [128]:
torch.max(outputs.data, 1)

torch.return_types.max(
values=tensor([37.3564, 40.4309, 37.0757, 37.2045], device='cuda:0'),
indices=tensor([3, 4, 3, 1], device='cuda:0'))

### inference

In [420]:
# train all by one tensor / max_score
clip_model, preprocess = clip.load("ViT-B/32", device=device)
model = OurCLIP(clip_model, 
                trainable_projection1=True, trainable_projection2=True,
                score_type=0).to(device)  # model을 initialize하는 부분

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.1)

num_epochs = 3
for epoch in range(num_epochs):  # loop over the dataset multiple times
    print("------------------ Training Epoch {} ------------------".format(epoch+1))
    train_one_epoch(model, optimizer, train_loader, device, prompts)

    test_model(model, test_loader)

print('Finished Training')

------------------ Training Epoch 1 ------------------
[    0] loss: 1.482, speed: 0.05, accuracy: 100.00 %
[   10] loss: 14.675, speed: 0.55, accuracy: 90.00 %
[   20] loss: 14.948, speed: 0.99, accuracy: 100.00 %
[   30] loss: 14.323, speed: 1.36, accuracy: 100.00 %
[   40] loss: 14.897, speed: 1.69, accuracy: 100.00 %
[   50] loss: 15.437, speed: 1.98, accuracy: 100.00 %
[   60] loss: 14.796, speed: 2.25, accuracy: 80.00 %
[   70] loss: 15.336, speed: 2.48, accuracy: 100.00 %
[   80] loss: 15.057, speed: 2.69, accuracy: 100.00 %
[   90] loss: 14.951, speed: 2.89, accuracy: 100.00 %
[  100] loss: 14.955, speed: 3.06, accuracy: 90.00 %
[  110] loss: 14.965, speed: 3.22, accuracy: 100.00 %
[  120] loss: 15.409, speed: 3.36, accuracy: 90.00 %
[  130] loss: 14.486, speed: 3.49, accuracy: 100.00 %
[  140] loss: 14.172, speed: 3.61, accuracy: 100.00 %
[  150] loss: 15.129, speed: 3.73, accuracy: 100.00 %
[  160] loss: 14.910, speed: 3.84, accuracy: 100.00 %
[  170] loss: 14.614, speed: 3.9

KeyboardInterrupt: 

In [404]:
# train all by one tensor / mean_score
clip_model, preprocess = clip.load("ViT-B/32", device=device)
model = OurCLIP(clip_model, 
                trainable_projection1=True, trainable_projection2=True,
                score_type=1).to(device)  # model을 initialize하는 부분

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.1)

num_epochs = 2

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print("------------------ Training Epoch {} ------------------".format(epoch+1))
    train_one_epoch(model, optimizer, train_loader, device, prompts)

    test_model(model, test_loader)

print('Finished Training')

------------------ Training Epoch 1 ------------------
[    0] loss: 1.957, speed: 0.06, accuracy: 0.00 %
[   10] loss: 19.352, speed: 0.62, accuracy: 10.00 %
[   20] loss: 19.253, speed: 1.10, accuracy: 30.00 %
[   30] loss: 19.116, speed: 1.50, accuracy: 10.00 %
[   40] loss: 19.115, speed: 1.86, accuracy: 20.00 %
[   50] loss: 19.086, speed: 2.17, accuracy: 10.00 %
[   60] loss: 19.234, speed: 2.44, accuracy: 20.00 %
[   70] loss: 19.171, speed: 2.68, accuracy: 10.00 %
[   80] loss: 19.221, speed: 2.90, accuracy: 0.00 %
[   90] loss: 19.153, speed: 3.10, accuracy: 20.00 %
[  100] loss: 19.284, speed: 3.28, accuracy: 0.00 %
[  110] loss: 19.204, speed: 3.44, accuracy: 10.00 %
[  120] loss: 19.315, speed: 3.59, accuracy: 20.00 %
[  130] loss: 19.096, speed: 3.72, accuracy: 10.00 %
[  140] loss: 19.288, speed: 3.85, accuracy: 0.00 %
[  150] loss: 19.274, speed: 3.97, accuracy: 0.00 %
[  160] loss: 19.367, speed: 4.07, accuracy: 10.00 %
[  170] loss: 19.269, speed: 4.17, accuracy: 30.00

In [419]:
# train all by each tensor / mean_score
clip_model, preprocess = clip.load("ViT-B/32", device=device)
model = OurCLIP(clip_model, 
                use_one_projection1=False, use_one_projection2=False,
                trainable_projection1=True, trainable_projection2=True,
                score_type=0).to(device)  # model을 initialize하는 부분

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.1)

num_epochs = 3

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print("------------------ Training Epoch {} ------------------".format(epoch+1))
    train_one_epoch(model, optimizer, train_loader, device, prompts)

    test_model(model, test_loader)

print('Finished Training')

------------------ Training Epoch 1 ------------------
[    0] loss: 1.944, speed: 0.08, accuracy: 0.00 %
[   10] loss: 19.566, speed: 0.76, accuracy: 0.00 %
[   20] loss: 19.425, speed: 1.32, accuracy: 0.00 %
[   30] loss: 19.506, speed: 1.78, accuracy: 0.00 %
[   40] loss: 19.524, speed: 2.17, accuracy: 0.00 %
[   50] loss: 19.489, speed: 2.51, accuracy: 0.00 %
[   60] loss: 19.412, speed: 2.80, accuracy: 0.00 %
[   70] loss: 19.386, speed: 3.05, accuracy: 0.00 %
[   80] loss: 19.382, speed: 3.27, accuracy: 0.00 %
[   90] loss: 19.489, speed: 3.47, accuracy: 0.00 %
[  100] loss: 19.463, speed: 3.65, accuracy: 0.00 %
[  110] loss: 19.475, speed: 3.81, accuracy: 0.00 %
[  120] loss: 19.470, speed: 3.95, accuracy: 0.00 %
[  130] loss: 19.506, speed: 4.08, accuracy: 0.00 %
[  140] loss: 19.424, speed: 4.20, accuracy: 0.00 %
[  150] loss: 19.535, speed: 4.30, accuracy: 20.00 %
[  160] loss: 19.492, speed: 4.41, accuracy: 0.00 %
[  170] loss: 19.353, speed: 4.50, accuracy: 0.00 %
[  180] l

In [423]:
clip_model.visual.proj, model.visual_projection.requires_grad

(Parameter containing:
 tensor([[-2.6264e-03,  5.0962e-05,  2.7496e-02,  ..., -1.0025e-02,
          -1.2222e-02,  5.8403e-03],
         [-1.9852e-02,  7.1182e-03,  8.9788e-04,  ...,  1.1528e-02,
          -1.9485e-02, -8.0185e-03],
         [-8.6288e-03,  1.9226e-03, -2.1725e-03,  ...,  3.9330e-03,
          -1.1269e-02,  1.5345e-03],
         ...,
         [-1.1993e-02,  1.2955e-02,  2.5848e-02,  ..., -9.8038e-03,
          -4.2076e-03,  1.5211e-04],
         [-1.2871e-02, -9.5673e-03, -1.0826e-02,  ..., -7.0610e-03,
          -4.3182e-03, -4.9353e-04],
         [-4.4098e-03,  3.3588e-03, -1.2054e-02,  ...,  6.1073e-03,
           3.9940e-03, -3.0861e-03]], device='cuda:0', dtype=torch.float16,
        requires_grad=True),
 tensor([[-2.6264e-03,  5.0962e-05,  2.7496e-02,  ..., -1.0025e-02,
          -1.2222e-02,  5.8403e-03],
         [-1.9852e-02,  7.1182e-03,  8.9788e-04,  ...,  1.1528e-02,
          -1.9485e-02, -8.0185e-03],
         [-8.6288e-03,  1.9226e-03, -2.1725e-03,  ..., 

In [ ]:
# zeroshot
def test_model_for_clip_model(clip_model, data_loader):

    clip_model.eval()

    start_time = time.time()
    total = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['images']
            labels = torch.squeeze(data['labels'])
        
            inputs = inputs.to(device)
            labels = labels.to(device)

            # inputs = inputs.permute([0, 3, 1, 2])   # N, H, W, C -> N, C, H, W

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = clip_model(inputs, prompts)[0].softmax(dim=-1)
    
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
            
        print('Finished Testing')
        print('Testing accuracy: %.1f %%' %(accuracy))

In [ ]:
test_model_for_clip_model(clip_model, test_loader)